In [1]:
import numpy as np
import torch
from src.Data_fetching import get_pvgis_hourly
from src.Tensors import Tensors

import matplotlib.pyplot as plt

In [2]:
# Data fetching
latitude =52
longitude = 0
optimal_angles = 1                                                                    # 1 if azimuth and tilt are not known, otherwise 0
azimuth = None                                                                        # None if azimuth is not known
tilt = None                                                                           # None if tilt is not known
peak_power = 2.48                                                                     # The peak output of the system
loss = 14                        

In [3]:
start_date = 2005
end_date = None

parameters_PVGIS = {'lat': latitude,
              'lon': longitude,
              'startyear': start_date,
              'endyear': end_date,
              'outputformat': 'json',
              'angle': tilt,
              'aspect': azimuth,
              'optimalangles': optimal_angles,
              'pvcalculation': 1,
              'components': 1,
              'peakpower': peak_power,
              'loss': loss,
              'localtime': 0}

In [4]:
pv_data = get_pvgis_hourly(parameters_PVGIS)[['P']]

In [5]:
pv_data.index = pv_data.index.floor('h').tz_localize(None)

In [6]:
pv_data.loc[:, 'hour_sin'] = np.sin(2 * np.pi * pv_data.index.hour / 24)
pv_data.loc[:, 'hour_cos'] = np.cos(2 * np.pi * pv_data.index.hour / 24)

In [7]:
past_features = ['P']                                                                 
future_features = ['hour_sin',                                                        
                   'hour_cos']

lags = 24                                                                             
forecast_period = 24                                                                  
gap = -23
forecast_gap = -23

In [8]:
# Get the data in the torch.tensor format
target_tensors = Tensors(pv_data,
                         'P',
                         past_features ,
                         future_features,
                         lags,
                         forecast_period,
                         gap=gap,
                         forecast_gap=forecast_gap,
                         train_test_split=0.8)

# Split the data into train and test sets with separate tensors for features (X) and the target (y)
X_train, X_test, y_train, y_test = target_tensors.create_tensor()
X_cal = X_train[-X_test.shape[0]:,:,:]
X_train = X_train[:-X_test.shape[0],:,:]
y_cal = y_train[-y_test.shape[0]:,:]
y_train = y_train[:-y_test.shape[0],:]

X_train.shape, X_cal.shape, X_test.shape, y_train.shape, y_cal.shape, y_test.shape

(torch.Size([84140, 24, 3]),
 torch.Size([28046, 24, 3]),
 torch.Size([28046, 24, 3]),
 torch.Size([84140, 24]),
 torch.Size([28046, 24]),
 torch.Size([28046, 24]))

In [9]:
from src.DeepAR import DeepAR
from src.DeepAR import Training

In [10]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [11]:
deepar = DeepAR(input_size=3,
                hidden_size=200,
                num_layers=2,
                dropout=0.5,
                )

In [12]:
trainer = Training(deepar,X_train,y_train, X_test, y_test, epochs=5,batch_size=64, learning_rate=0.001)

In [13]:
trainer.train()

Epoch 1/5, Loss: -2.1712
Epoch 2/5, Loss: -2.3647
Epoch 3/5, Loss: -2.3271
Epoch 4/5, Loss: -2.0834
Epoch 5/5, Loss: -2.5326


In [21]:
deepar.eval()

DeepAR(
  (lstm): LSTM(3, 200, num_layers=2, batch_first=True, dropout=0.5)
  (linear): Linear(in_features=200, out_features=2, bias=True)
)

In [25]:
mu, sigma = deepar(X_test[0,:,:].to(device))

In [28]:
mu

tensor([1.8025e-01, 2.4900e-01, 2.0504e-01, 2.7979e-01, 2.4177e-01, 1.2898e-01,
        8.7162e-02, 3.9731e-02, 8.2706e-03, 1.0376e-03, 4.6918e-04, 3.4323e-04,
        2.6353e-04, 3.3738e-04, 4.2231e-04, 2.5366e-04, 6.8411e-05, 6.9524e-04,
        9.4481e-03, 6.4024e-02, 1.1583e-01, 2.0265e-01, 3.5086e-01, 3.0221e-01],
       device='cuda:0', grad_fn=<SelectBackward0>)

In [31]:
sample = np.random.normal(mu.cpu().detach().numpy(),sigma.cpu().detach().numpy())

In [32]:
sample

array([ 2.14978731e-01,  4.73519715e-01,  4.29810834e-01,  3.52257085e-01,
        2.76208712e-01,  1.44893300e-01,  1.75387775e-01,  1.92023257e-02,
        5.08257169e-03,  9.38251584e-05, -1.40495124e-03, -3.42707062e-03,
        1.43032538e-03,  7.18124336e-04,  2.14474090e-04, -7.96724285e-04,
       -1.03670855e-03, -2.05631230e-03,  3.22301677e-03,  3.15597709e-02,
        1.96750555e-01,  3.01320770e-01,  2.80677238e-01, -2.40630814e-02])